# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at mlta-2022-spring@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1H5ZONrb2LMOCixLY7D5_5-7LkIaXO6AGEaV2mRdTOMY/edit?usp=sharing)　Kaggle: [Link](https://www.kaggle.com/c/ml2022spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2.5hrs
  

## Download Dataset

In [ ]:
# Download link 1
!gdown --id '1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1qwjbRjq481lHsnTrrF4OjKQnxzgoLEFR' --output hw7_data.zip

# Download Link 3 (if the above link fails) 
# !gdown --id '1QXuWjNRZH6DscSd6QcRER0cnxmpZvijn' --output hw7_data.zip

!unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [ ]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

## Import Packages

In [1]:
import json
import numpy as np
import random
import torch
import os
from torch.utils.data import DataLoader, Dataset 
from transformers import BertForQuestionAnswering, BertTokenizerFast

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(0)


In [2]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

## Load Tokenizer

In [3]:
tokenizer = BertTokenizerFast.from_pretrained("luhua/chinese_pretrain_mrc_macbert_large")

## Read Data

- Training set: 31690 QA pairs
- Dev set: 4131  QA pairs
- Test set: 4957  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [4]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [5]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

In [6]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs, max_seq_len=512, doc_stride=256):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = max_seq_len
        self.max_question_len = 40
        self.max_paragraph_len = max_seq_len - (1 + self.max_question_len + 1 + 1)
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = doc_stride       # Overlapping

        

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            mid = int(random.uniform(answer_start_token, answer_end_token))
            # mid = int((answer_start_token + answer_end_token) // (2 + random.uniform(-1, 1)))
            # mid = (answer_start_token + answer_end_token) // 2
            paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len
            
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

## Configuration

In [7]:
max_seq_len = 512
doc_stride = 256
topk = 5

## DataLoader

In [8]:
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized, max_seq_len=max_seq_len, doc_stride=doc_stride)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized, max_seq_len=max_seq_len, doc_stride=doc_stride)

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Testing

In [9]:
models_path = ['./models/mrc-macbert-qa', './models/mrc-roberta-qa', './models/macbert', './models/roberta', './models/uer']
for path in models_path:
    model_name = path.split('/')[-1]
    # If model is not exist, run the python script.
    if not os.path.exists(path):
        os.system(f"python {model_name}-large.py -p {model_name}")
    else:
        print(f"Model {model_name} exits.")

# Load models
models = []
for path in models_path:
    model = BertForQuestionAnswering.from_pretrained(path).to(device)
    models.append(model)

Start Training ...


 25%|██▌       | 998/3962 [04:56<14:33,  3.39it/s]

Epoch 1 | Step 1000 | loss = 0.642, acc = 0.750


 50%|█████     | 1998/3962 [09:56<09:42,  3.37it/s]

Epoch 1 | Step 2000 | loss = 0.518, acc = 0.800


...

Epoch 3 | Step 3000 | loss = 0.065, acc = 0.964


  0%|          | 0/4131 [00:00<?, ?it/s]

Evaluating Dev Set ...


100%|██████████| 4131/4131 [02:37<00:00, 26.28it/s]


Validation | Epoch 3 | acc = 0.807


In [10]:
def evaluate_ensemble(data, outputs, doc_stride, paragraph, paragraph_offset, topk=5):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''                         # Prediction answer.
    max_prob = float('-inf')            # Maximum probability of logits.
    num_of_windows = data[0].shape[1]   # Total windows.
    window = 0                          # Index of window with max probability.
    win_context = None                  # Ids list of window.
    start_i, end_i = 0, 0               # (Start, End) of window with max probability.
    new_start, new_end = 0, 0           # New (start, end) pair in original paragraph.

    for k in range(num_of_windows):
        # Base index after the Question [CLS] Question? [SEP]
        paragraph_base = list(data[0][0][k]).index(102) + 1
        # Last index of [SEP] in [SEP] paragraph... [SEP]
        paragraph_sep = len(list(data[0][0][k])) - 1 - list(data[0][0][k])[::-1].index(102)

        # Mean of logits
        start_logits, end_logits = 0, 0
        for output in outputs:
            start_logits += output.start_logits
            end_logits += output.end_logits
        start_logits, end_logits = start_logits / len(outputs), end_logits / len(outputs)

        # Keep the most probable top k start position / end position
        start_probs, start_indices = torch.topk(start_logits[k], topk, dim=0)
        end_probs, end_indices = torch.topk(end_logits[k], topk, dim=0)

        # Probability of answer is calculated as sum of start_prob and end_prob
        # (start_index, end_index) must in the range of [paragraph_base, paragraph_sep)
        # Length of answer is less than 30
        prob = float('-inf')
        for start_prob, start_idx in zip(start_probs, start_indices):
            for end_prob, end_idx in zip(end_probs, end_indices):
                if (start_idx <= end_idx) and (end_idx - start_idx <= 30) and (start_idx >= paragraph_base) and (end_idx < paragraph_sep):
                    if start_prob + end_prob > prob:
                        prob = start_prob + end_prob
                        start_index, end_index = start_idx, end_idx
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
            win_context = data[0][0][k]
            start_i, end_i = start_index - paragraph_base, end_index - paragraph_base + 1
            window = k

    # '[UNK]' in prediction answer.
    if '[UNK]' in answer:
        print(f"[UNK] in prediction answer\n{answer}")
        print(f"Prediction range: [{start_i}, {end_i}) in window {window}-th paragraph")

    # Calculate the (start, end) pair corresponding to original paragraph.
    for idx in range(start_i + window * doc_stride, end_i + window * doc_stride):
        sub_start, sub_end = paragraph_offset[idx]
        if idx == start_i + window * doc_stride: 
            new_start = sub_start
        if idx == end_i + window * doc_stride - 1:
            new_end = sub_end

    if '「' in paragraph[new_start:new_end] and paragraph[new_end - 1] != '」':
        new_end += 1
    elif paragraph[new_start] != '「' and '」' in paragraph[new_start:new_end]:
        new_start -= 1
    elif '《' in paragraph[new_start:new_end] and paragraph[new_end - 1] != '》':
        new_end += 1
    elif paragraph[new_start] != '《' and '》' in paragraph[new_start:new_end]:
        new_start -= 1

    if '[UNK]' in answer:
        print(f"Original answer\n{paragraph[new_start:new_end]}")

    # Get answer from original paragraph.
    answer = paragraph[new_start:new_end]
    return answer

## Ensembles on Validation dataset

In [11]:
print("Evaluating Dev Set ...")
model.eval()
with torch.no_grad():
    dev_acc = 0
    for i, data in enumerate(tqdm(dev_loader)):
        outputs = []
        for model in models:
            output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
            outputs.append(output)
        # prediction is correct only if answer text exactly matches
        predict_ans = evaluate_ensemble(data, outputs, doc_stride, dev_paragraphs[dev_questions[i]['paragraph_id']], dev_paragraphs_tokenized[dev_questions[i]['paragraph_id']].offsets, topk)
        golden_ans = dev_questions[i]["answer_text"]
        if predict_ans != golden_ans:
            print(f"Question: {dev_questions[i]['question_text']}\nGolden Answer: {golden_ans}")
            print(f"Your   Answer: {predict_ans}")
        dev_acc += predict_ans == golden_ans
    print(f"Validation | acc = {dev_acc / len(dev_loader):.3f}")

Evaluating Dev Set ...


  0%|          | 12/4131 [00:02<15:42,  4.37it/s]

Question: 中國唯一以「郵」命名的城市指的是哪一個城市?
Golden Answer: 高郵
Your   Answer: 高郵市
Question: 長江是世界第幾長河?
Golden Answer: 三
Your   Answer: 第三長河


  0%|          | 14/4131 [00:03<16:21,  4.20it/s]

Question: 梁啟超認為合作勤王計畫失敗的原因是?
Golden Answer: 康有為故意不發軍餉造成
Your   Answer: 擅作主張分散兵力


...

Question: 哥倫比亞和厄瓜多及委內瑞拉共享什麼山脈地區?
Golden Answer: 安第斯
Your   Answer: 安第斯山脈


100%|█████████▉| 4127/4131 [15:01<00:00,  5.01it/s]

Question: 哥倫比亞是世界面積第幾大國家?
Golden Answer: 26
Your   Answer: 第26大


100%|██████████| 4131/4131 [15:02<00:00,  4.58it/s]

Validation | acc = 0.839


## Ensembles on Testing dataset

In [12]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        outputs = []
        for model in models:
            output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
            outputs.append(output)
        result.append(evaluate_ensemble(data, outputs, doc_stride, test_paragraphs[test_questions[i]['paragraph_id']], test_paragraphs_tokenized[test_questions[i]['paragraph_id']].offsets, topk))

result_file = f"result-ensemble-top5-mix{topk}-30.csv"
with open(result_file, 'w') as f:
    f.write("ID,Answer\n")
    for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
        f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  1%|          | 51/4957 [00:11<17:05,  4.78it/s]

[UNK] in prediction answer
大 型 購 物 中 心 [UNK] 開 幕
Prediction range: [161, 170) in window 0-th paragraph
Original answer
大型購物中心MegaBox開幕


  5%|▌         | 252/4957 [00:58<17:20,  4.52it/s]

[UNK] in prediction answer
溥 [UNK]
Prediction range: [392, 394) in window 0-th paragraph
Original answer
溥儁


...

[UNK] in prediction answer
白 [UNK] 紀
Prediction range: [115, 118) in window 0-th paragraph
Original answer
白堊紀


 73%|███████▎  | 3608/4957 [13:40<04:23,  5.11it/s]

[UNK] in prediction answer
白 [UNK] 紀
Prediction range: [0, 3) in window 0-th paragraph
Original answer
白堊紀


100%|██████████| 4957/4957 [18:38<00:00,  4.43it/s]

Completed! Result is in result-ensemble-top5-mix5-30.csv


### Public Accuracy: 0.85155